# Importações


In [1]:
import urllib
from itertools import product
from os import getenv
from sqlalchemy import create_engine
from dotenv import load_dotenv

import numpy as np
import pandas as pd
from datetime import datetime, timedelta, time
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
from fuzzywuzzy import process
from enum import Enum

### Types


In [2]:
# cSpell: disable
class BSColorsEnum(Enum):

    DANGER_COLOR = "#dc3545"

    WARNING_COLOR = "#ffc107"

    SUCCESS_COLOR = "#198754"

    GREY_500_COLOR = "#adb5bd"

    GREY_600_COLOR = "#6c757d"

    GREY_700_COLOR = "#495057"

    GREY_800_COLOR = "#343a40"

    GREY_900_COLOR = "#212529"

    PRIMARY_COLOR = "#0d6efd"

    SECONDARY_COLOR = "#6c757d"

    INFO_COLOR = "#0dcaf0"

    GRAY_COLOR = "#adb5bd"

    TEAL_COLOR = "#20c997"

    ORANGE_COLOR = "#fd7e14"

    INDIGO_COLOR = "#6610f2"

    PINK_COLOR = "#d63384"

    PURPLE_COLOR = "#6f42c1"

    GREY_400_COLOR = "#ced4da"

    SPACE_CADET_COLOR = "#282f44"

    BLUE_DELFT_COLOR = "#0d6efd"


class IndicatorType(Enum):
    PERFORMANCE = "performance"
    REPAIR = "reparo"
    EFFICIENCY = "eficiencia"

# Database


## Conexão com o banco de dados


In [3]:
# database/connection.py

# cSpell: disable=invalid-name
load_dotenv()


class Connection:
    """
    Class Connection
    """

    def __init__(self):
        """
        Constructor

        Args:
            user (str): user
            password (str): password
            database (str): database
            driver (str): driver
            server (str): server

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        self.__user = getenv("PYMSSQL_USER")
        self.__password = getenv("PYMSSQL_PASSWORD")
        self.__database = getenv("PYMSSQL_DATABASE_AUTOMACAO")
        self.__database_totvsdb = getenv("PYMSSQL_DATABASE_TOTVSDB")
        self.__driver = "{ODBC Driver 17 for SQL Server}"
        self.__server = getenv("PYMSSQL_SERVER")

    def get_connection_automacao(self):
        """
        Get connection

        Returns:
            object: connection

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        try:
            params = urllib.parse.quote_plus(
                f"DRIVER={self.__driver};"
                f"SERVER={self.__server};"
                f"DATABASE={self.__database};"
                f"UID={self.__user};"
                f"PWD={self.__password};"
            )
            # pylint: disable=consider-using-f-string
            conexao_automacao = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
            return conexao_automacao
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

    def get_connection_totvsdb(self):
        """
        Get connection

        Returns:
            object: connection

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        try:
            params = urllib.parse.quote_plus(
                f"DRIVER={self.__driver};"
                f"SERVER={self.__server};"
                f"DATABASE={self.__database_totvsdb};"
                f"UID={self.__user};"
                f"PWD={self.__password};"
            )
            conexao_totvsdb = create_engine(f"mssql+pyodbc:///?odbc_connect={params}", pool_size=2)
            return conexao_totvsdb
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

## Leitura do banco de dados


In [4]:
# database/db_read.py


# cSpell: disable=invalid-name
class Read(Connection):
    """
    Class Read
    Read data from the database and return a pandas dataframe
    Create query to be executed in the database
    """

    # pylint: disable=useless-super-delegation
    def __init__(self):
        """
        Constructor
        """
        super().__init__()

    def get_automacao_data(self, query: str) -> pd.DataFrame:
        """
        Get data from database AUTOMACAO and return a pandas dataframe.

        Parameters
        ----------
        query : str
            Query to be executed in the database

        Returns
        -------
        pandas dataframe
            Dataframe with the query result
        """
        connection = None
        try:
            connection = self.get_connection_automacao()
            data = pd.read_sql(query, connection)
            return data
        # pylint: disable=broad-except
        except Exception as e:
            print(f"Erro ao buscar dados: {e}")
            return None
        finally:
            if connection:
                connection.dispose()

    def create_automacao_query(self, table: str, where: str = None, orderby: str = None) -> str:
        """
        Create query to be executed in the database AUTOMACAO.

        Parameters
        ----------
        table : str
            Table name
        where : str
            Where clause (optional)
        orderby : str
            Order by clause (optional)

        Returns
        -------
        str
            Query to be executed in the database
        """
        query = f"SELECT * FROM AUTOMACAO.dbo.{table}"

        if where:
            query += f" WHERE {where}"

        if orderby:
            query += f" ORDER BY {orderby}"

        return query

    def get_totvsdb_data(self, query: str) -> pd.DataFrame:
        """
        Retrieves data from the TotvsDB database using the provided SQL query.

        Args:
            query (str): The SQL query to execute.

        Returns:
            pd.DataFrame: A pandas DataFrame containing the retrieved data.

        Raises:
            Exception: If an error occurs while retrieving the data.

        """
        connection = None
        try:
            connection = self.get_connection_totvsdb()
            data = pd.read_sql(query, connection)
            return data
        # pylint: disable=broad-except
        except Exception as e:
            print(f"Erro ao buscar dados: {e}")
            return None
        finally:
            if connection:
                connection.dispose()

    def create_totvsdb_query(
        self, select: str, table: str, join: str = None, where: str = None, orderby: str = None
    ) -> str:
        """
        Creates a SQL query string for querying the TOTVS database.

        Args:
            select (str): The SELECT clause of the query.
            table (str): The table name to query.
            join (str, optional): The JOIN clause of the query. Defaults to None.
            where (str, optional): The WHERE clause of the query. Defaults to None.
            orderby (str, optional): The ORDER BY clause of the query. Defaults to None.

        Returns:
            str: The SQL query string.
        """
        query = f"SELECT {select} FROM {table}"

        if join:
            query += f" {join}"

        if where:
            query += f" WHERE {where}"

        if orderby:
            query += f" ORDER BY {orderby}"

        return query

## Query para o banco de dados


In [17]:
# // database/get_data.py
# cSpell: disable=invalid-name
class GetData:
    """
    Essa classe é responsável por realizar a leitura dos dados do banco de dados.
    É utilizada para fazer a leitura em segundo plano, sem que o usuário perceba.
    """

    def __init__(self):
        self.db_read = Read()

    def get_data(self) -> pd.DataFrame:

        select = (
            'T1.CYV_CDMQ AS "Codigo_Maquina"'
            ', T2.CYB_DSMQ AS "Descricao_Maquina"'
            ', T1.CYV_QTATRP AS "Quantidade_Atropelamento"'
            ', T1.CYV_DTRPBG AS "Data_Registro"'
            ', T1.CYV_HRRPBG AS "Hora_Registro"'
            ', T1.CYV_CDUSRP AS "Usuario_Registro"'
            ", COALESCE("
            "CASE WHEN CHARINDEX(T1.CYV_CDUSRP, T3.X6_CONTEUD) > 0 THEN 'Fabrica 1' END,"
            "CASE WHEN CHARINDEX(T1.CYV_CDUSRP, T4.X6_CONTEUD) > 0 THEN 'Fabrica 2' END,"
            "'Não identificado'"
            ') AS "Fabrica"'
        )

        table = "CYV000 (NOLOCK) AS T1"

        join = (
            "JOIN CYB000 (NOLOCK) AS T2 ON T1.CYV_FILIAL = T2.CYB_FILIAL AND T1.CYV_CDMQ = T2.CYB_CDMQ AND T2.D_E_L_E_T_ <> '*'"
            "LEFT JOIN SX6000 (NOLOCK) AS T3 ON T3.X6_VAR = 'MV_X_USRF1'"
            "LEFT JOIN SX6000 (NOLOCK) AS T4 ON T4.X6_VAR = 'MV_X_USRF2'"
        )

        where = "T1.D_E_L_E_T_ <> '*' AND T1.CYV_FILIAL = '0101' AND T1.CYV_CDMQ LIKE 'AMS%' AND T1.CYV_DTRPBG BETWEEN '20240430' AND '20240601'"

        orderby = "T1.CYV_DTRPBG, T1.CYV_CDMQ, T1.CYV_HRRPBG"

        # Query para buscar os dados de Massadas
        query_massadas = self.db_read.create_totvsdb_query(select, table, join, where, orderby)

        df_massadas = self.db_read.get_totvsdb_data(query_massadas)

        # Verificando se os dados foram lidos corretamente
        if df_massadas.empty:
            print("====== Erro na leitura dos dados ======")
            return None

        return df_massadas

    def get_production_by(self):
        query = (
            "WITH CTE AS ( "
            "SELECT "
            "maquina_id, turno, contagem_total_ciclos, contagem_total_produzido, "
            "produto_id, SB1.B1_DESC as produto, data_registro, hora_registro, "
            "ROW_NUMBER() OVER(PARTITION BY maquina_id, data_registro, turno, produto_id ORDER BY hora_registro) as rn, "
            "ROW_NUMBER() OVER(PARTITION BY maquina_id, data_registro, turno, produto_id ORDER BY hora_registro DESC) as rn_desc "
            "FROM ( "
            "SELECT "
            "(SELECT TOP 1 t2.fabrica FROM AUTOMACAO.dbo.maquina_cadastro t2 "
            "WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro "
            "ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as fabrica, "
            "(SELECT TOP 1 t2.linha FROM AUTOMACAO.dbo.maquina_cadastro t2 "
            "WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro "
            "ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as linha, "
            "t1.maquina_id, t1.turno, t1.contagem_total_ciclos, t1.contagem_total_produzido, "
            "(SELECT TOP 1 t2.produto_id FROM AUTOMACAO.dbo.maquina_produto t2 "
            "WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro "
            "ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as produto_id, "
            "t1.data_registro, "
            "t1.hora_registro "
            "FROM AUTOMACAO.dbo.maquina_info t1 "
            ") AS t "
            "INNER JOIN TOTVSDB.dbo.SB1000 SB1 WITH (NOLOCK) ON SB1.B1_FILIAL = '01' AND SB1.B1_COD = produto_id AND SB1.D_E_L_E_T_<>'*' "
            ") "
            "SELECT "
            "C1.maquina_id, C1.turno, C1.produto_id, C1.produto, C1.data_registro, "
            "MIN(C1.hora_registro) as hora_inicial, "
            "MAX(C1.hora_registro) as hora_final, "
            "MAX(C1.contagem_total_ciclos) - MIN(C1.contagem_total_ciclos) as total_ciclos, "
            "MAX(C1.contagem_total_produzido) - MIN(C1.contagem_total_produzido) as total_produzido "
            "FROM CTE C1 "
            "GROUP BY C1.maquina_id, C1.turno, C1.produto_id, C1.produto, C1.data_registro "
            "ORDER BY C1.maquina_id ASC, C1.data_registro DESC; "
        )

        df = pd.DataFrame()

        try:
            df = self.db_read.get_automacao_data(query)
        except Exception as e:
            print(f"Erro ao buscar dados: {e}")

        return df


get_data = GetData()

In [6]:
df_massadas = get_data.get_data()
df_massadas.head()

,Codigo_Maquina,Descricao_Maquina,Quantidade_Atropelamento,Data_Registro,Hora_Registro,Usuario_Registro,Fabrica
0,AMS001,AMASS ESPIRAL 01,153.00,20240501,16:00:00,000371,Fabrica 1
1,AMS002,AMASS ESPIRAL 02,114.75,20240501,16:30:00,000371,Fabrica 1
2,AMS002,AMASS ESPIRAL 02,153.00,20240501,18:13:59,000276,Fabrica 1
3,AMS002,AMASS ESPIRAL 02,153.00,20240501,18:14:09,000276,Fabrica 1
4,AMS002,AMASS ESPIRAL 02,153.00,20240501,18:14:18,000276,Fabrica 1


In [18]:
df_prod_by_product = get_data.get_production_by()
df_prod_by_product.head()

,maquina_id,turno,produto_id,produto,data_registro,hora_inicial,hora_final,total_ciclos,total_produzido
0,TMF001,NOT,0002,PAO ALHO PIC 10B/400GR ...,2024-06-05,00:00:54.256666,07:58:55.613333,188.0,0.0
1,TMF001,MAT,0002,PAO ALHO PIC 10B/400GR ...,2024-06-05,08:00:55.616666,14:52:56.780000,112.0,0.0
2,TMF001,NOT,0002,PAO ALHO PIC 10B/400GR ...,2024-06-04,00:00:50.206666,07:58:51.570000,118.0,0.0
3,TMF001,MAT,0002,PAO ALHO PIC 10B/400GR ...,2024-06-04,08:00:51.570000,15:58:52.923333,6112.0,0.0
4,TMF001,VES,0002,PAO ALHO PIC 10B/400GR ...,2024-06-04,16:00:52.933333,23:58:54.250000,2350.0,0.0


In [7]:
MASSADA_CHEIA = 153
MASSADA_REPROCESSO = 114.75
MASSADA_BOLINHA = 77
RENDIMENTO_CHEIA = 2240
RENDIMENTO_REPROCESSO = 1680
RENDIMENTO_BOLINHA = 4000

In [8]:
# cSpell: disable=invalid-name
class CleanData:
    def __init__(self) -> None:
        pass

    def _get_shift(self, time: time) -> str:
        if (
            time >= datetime.strptime("08:00:00", "%H:%M:%S").time()
            and time < datetime.strptime("16:00:00", "%H:%M:%S").time()
        ):
            return "MAT"
        elif (
            time >= datetime.strptime("16:00:00", "%H:%M:%S").time()
            and time < datetime.strptime("23:59:59", "%H:%M:%S").time()
        ):
            return "VES"
        else:
            return "NOT"

    def _get_massadas_total(self, df: pd.DataFrame, position: int = None) -> pd.DataFrame:
        # Soma os valores por maquina, data e turno
        df_massadas_total = (
            df.groupby(["Codigo_Maquina", "Descricao_Maquina", "Data_Registro", "Turno", "Fabrica"])
            .agg(
                Usuario_Registro=("Usuario_Registro", "first"),
                Batidas_Cheia=("Quantidade_Atropelamento", "count"),
                Peso_Massa_BC=("Quantidade_Atropelamento", "sum"),
            )
            .reset_index()
        )

        # Cria um dict para renomear colunas
        rename_columns = {
            1: {
                "Batidas_Cheia": "Batidas_Reprocesso",
                "Peso_Massa_BC": "Peso_Massa_BR",
            },
            2: {
                "Batidas_Cheia": "Batidas_Bolinha",
                "Peso_Massa_BC": "Peso_Massa_BB",
            },
        }

        # Renomeia as colunas
        if position is not None:
            df_massadas_total.rename(columns=rename_columns[position], inplace=True)

        return df_massadas_total

    def clean_massadas_data(self, df: pd.DataFrame) -> pd.DataFrame:
        # Atribuir coluna turno com base no horário
        df["Hora_Registro"] = pd.to_datetime(df["Hora_Registro"], format="%H:%M:%S").dt.time
        df["Turno"] = df["Hora_Registro"].apply(self._get_shift)

        # Separando o dataframe conforme a quantidade de atropelamentos
        df_massadas_cheias = df[df["Quantidade_Atropelamento"] == 153]
        df_massadas_reprocesso = df[df["Quantidade_Atropelamento"] == 114.75]
        df_massadas_bolinha = df[df["Quantidade_Atropelamento"] == 77]

        # Soma os valores por maquina, data e turno
        df_massadas_cheias = self._get_massadas_total(df_massadas_cheias)
        df_massadas_reprocesso = self._get_massadas_total(df_massadas_reprocesso, 1)
        df_massadas_bolinha = self._get_massadas_total(df_massadas_bolinha, 2)

        # Unir os dataframes de massadas cheias e reprocesso
        df_massadas = df_massadas_cheias.merge(
            df_massadas_reprocesso,
            how="outer",
            on=[
                "Codigo_Maquina",
                "Descricao_Maquina",
                "Data_Registro",
                "Turno",
                "Fabrica",
                "Usuario_Registro",
            ],
        )

        # Unir o dataframe de massadas com o dataframe de bolinha
        df_massadas = df_massadas.merge(
            df_massadas_bolinha,
            how="outer",
            on=[
                "Codigo_Maquina",
                "Descricao_Maquina",
                "Data_Registro",
                "Turno",
                "Fabrica",
                "Usuario_Registro",
            ],
        )

        # Tranformar data_registro em datetime no formato ano-mes-dia
        df_massadas["Data_Registro"] = pd.to_datetime(df_massadas["Data_Registro"], format="%Y%m%d")

        return df_massadas

    def clean_production_data(self, df: pd.DataFrame) -> pd.DataFrame:
        


# maquina_id	turno	produto_id	produto	data_registro	hora_inicial	hora_final	total_ciclos	total_produzido
clean_data = CleanData()

In [9]:
df_massadas_cleaned = clean_data.clean_massadas_data(df_massadas.copy())
df_massadas_cleaned.head(50)

,Codigo_Maquina,Descricao_Maquina,Data_Registro,Turno,Fabrica,Usuario_Registro,Batidas_Cheia,Peso_Massa_BC,Batidas_Reprocesso,Peso_Massa_BR,Batidas_Bolinha,Peso_Massa_BB
0,AMS00,AMASS ESPIRAL 00 - BOLINHA,2024-05-02,MAT,Fabrica 2,000316,NaN,NaN,NaN,NaN,1.0,77.0
1,AMS00,AMASS ESPIRAL 00 - BOLINHA,2024-05-09,MAT,Fabrica 1,000371,NaN,NaN,NaN,NaN,1.0,77.0
2,AMS00,AMASS ESPIRAL 00 - BOLINHA,2024-05-14,VES,Fabrica 2,000316,NaN,NaN,NaN,NaN,13.0,1001.0
3,AMS00,AMASS ESPIRAL 00 - BOLINHA,2024-05-15,MAT,Fabrica 2,000316,NaN,NaN,NaN,NaN,17.0,1309.0
4,AMS00,AMASS ESPIRAL 00 - BOLINHA,2024-05-15,NOT,Fabrica 2,000316,NaN,NaN,NaN,NaN,20.0,1540.0
5,AMS00,AMASS ESPIRAL 00 - BOLINHA,2024-05-15,VES,Fabrica 2,000316,NaN,NaN,NaN,NaN,6.0,462.0
6,AMS00,AMASS ESPIRAL 00 - BOLINHA,2024-05-19,VES,Fabrica 2,000316,NaN,NaN,NaN,NaN,4.0,308.0
7,AMS00,AMASS ESPIRAL 00 - BOLINHA,2024-05-20,MAT,Fabrica 2,000316,NaN,NaN,NaN,NaN,1.0,77.0
8,AMS00,AMASS ESPIRAL 00 - BOLINHA,2024-05-20,NOT,Fabrica 2,000316,NaN,NaN,NaN,NaN,17.0,1309.0
9,AMS00,AMASS ESPIRAL 00 - BOLINHA,2024-05-20,VES,Fabrica 2,000316,NaN,NaN,NaN,NaN,13.0,1001.0


In [10]:
# cSpell: disable=invalid-name
class dataAnalysis:
    def __init__(self):
        pass

    def get_agg_massa_data(self, df: pd.DataFrame) -> pd.DataFrame:
        df_massa_agg = (
            df.groupby(["Data_Registro", "Turno", "Fabrica"])
            .agg(
                Batidas_Cheia=("Batidas_Cheia", "sum"),
                Peso_Massa_BC=("Peso_Massa_BC", "sum"),
                Batidas_Reprocesso=("Batidas_Reprocesso", "sum"),
                Peso_Massa_BR=("Peso_Massa_BR", "sum"),
                Batidas_Bolinha=("Batidas_Bolinha", "sum"),
                Peso_Massa_BB=("Peso_Massa_BB", "sum"),
            )
            .reset_index()
        )
        return df_massa_agg

    def get_massa_paes_data(self, df: pd.DataFrame) -> pd.DataFrame:

        df["Baguete_Total"] = (
            df["Batidas_Cheia"] * RENDIMENTO_CHEIA
            + df["Batidas_Reprocesso"] * RENDIMENTO_REPROCESSO
        )
        df["Bolinha_Total"] = df["Batidas_Bolinha"] * RENDIMENTO_BOLINHA

        # Tornar as colunas em numeros inteiros
        columns_to_convert = [
            "Baguete_Total",
            "Bolinha_Total",
            "Batidas_Cheia",
            "Batidas_Reprocesso",
            "Batidas_Bolinha",
        ]
        df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, downcast="integer")

        # Ajustar o dia da semana para começar no domingo
        df["Dia_Semana"] = df["Data_Registro"].dt.weekday
        df["Dia_Semana"] = (df["Dia_Semana"] + 1) % 7

        # Criar uma nova coluna com a data da semana
        df["Data_Semana"] = df["Data_Registro"] - pd.to_timedelta(df["Dia_Semana"], unit="d")

        # Criar uma nova tabela com os dados agrupados por semana, fabrica e turno
        df_paes = (
            df.groupby(
                [
                    df["Data_Semana"].dt.isocalendar().year,
                    df["Data_Semana"].dt.isocalendar().week,
                    "Data_Semana",
                    "Turno",
                    "Fabrica",
                ]
            )
            .agg(
                Batidas_Cheia=("Batidas_Cheia", "sum"),
                Peso_Massa_BC=("Peso_Massa_BC", "sum"),
                Batidas_Reprocesso=("Batidas_Reprocesso", "sum"),
                Peso_Massa_BR=("Peso_Massa_BR", "sum"),
                Batidas_Bolinha=("Batidas_Bolinha", "sum"),
                Peso_Massa_BB=("Peso_Massa_BB", "sum"),
                Baguete_Total=("Baguete_Total", "sum"),
                Bolinha_Total=("Bolinha_Total", "sum"),
            )
            .reset_index()
        )

        return df, df_paes

In [11]:
df_massadas_agg = dataAnalysis().get_agg_massa_data(df_massadas_cleaned.copy())
df_massadas_agg.head(8)

,Data_Registro,Turno,Fabrica,Batidas_Cheia,Peso_Massa_BC,Batidas_Reprocesso,Peso_Massa_BR,Batidas_Bolinha,Peso_Massa_BB
0,2024-05-01,VES,Fabrica 1,54.0,8262.0,24.0,2754.00,0.0,0.0
1,2024-05-01,VES,Fabrica 2,2.0,306.0,2.0,229.50,9.0,693.0
2,2024-05-02,MAT,Fabrica 1,74.0,11322.0,21.0,2409.75,0.0,0.0
3,2024-05-02,MAT,Fabrica 2,1.0,153.0,4.0,459.00,23.0,1771.0
4,2024-05-02,NOT,Fabrica 1,58.0,8874.0,34.0,3901.50,0.0,0.0
5,2024-05-02,NOT,Fabrica 2,9.0,1377.0,5.0,573.75,13.0,1001.0
6,2024-05-02,VES,Fabrica 1,95.0,14535.0,36.0,4131.00,0.0,0.0
7,2024-05-02,VES,Fabrica 2,12.0,1836.0,6.0,688.50,18.0,1386.0


In [12]:
df_massadas_paes, semana_teste = dataAnalysis().get_massa_paes_data(df_massadas_agg.copy())
df_massadas_paes.head(8)
semana_teste

,year,week,Data_Semana,Turno,Fabrica,Batidas_Cheia,Peso_Massa_BC,Batidas_Reprocesso,Peso_Massa_BR,Batidas_Bolinha,Peso_Massa_BB,Baguete_Total,Bolinha_Total
0,2024,17,2024-04-28,MAT,Fabrica 1,192,29376.0,44,5049.00,0,0.0,504000,0
1,2024,17,2024-04-28,MAT,Fabrica 2,29,4437.0,4,459.00,43,3311.0,71680,172000
2,2024,17,2024-04-28,NOT,Fabrica 1,231,35343.0,65,7458.75,0,0.0,626640,0
3,2024,17,2024-04-28,NOT,Fabrica 2,39,5967.0,11,1262.25,48,3696.0,105840,192000
4,2024,17,2024-04-28,VES,Fabrica 1,233,35649.0,97,11130.75,0,0.0,684880,0
5,2024,17,2024-04-28,VES,Fabrica 2,34,5202.0,8,918.00,46,3542.0,89600,184000
6,2024,18,2024-05-05,MAT,Fabrica 1,437,66861.0,85,9753.75,1,77.0,1121680,4000
7,2024,18,2024-05-05,MAT,Fabrica 2,108,16524.0,15,1721.25,59,4543.0,267120,236000
8,2024,18,2024-05-05,NOT,Fabrica 1,554,84762.0,98,11245.50,0,0.0,1405600,0
9,2024,18,2024-05-05,NOT,Fabrica 2,122,18666.0,13,1491.75,53,4081.0,295120,212000
